In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import tqdm

In [2]:
articles_meta = pd.read_csv("data/cnn_articles_meta.csv")

In [3]:
keywords = ["covid", "coronavirus", "pandemic"]

In [4]:
covid_articles_meta = articles_meta[articles_meta.title.str.contains("|".join(keywords))]
covid_articles_meta

,title,date,url
5,'This is a time to come together': Pence defen...,2020-03-01,https://www.cnn.com/2020/03/01/politics/mike-p...
7,Paris' famed Louvre Museum is closed as France...,2020-03-01,https://www.cnn.com/2020/03/01/europe/france-c...
14,Biden slams Trump administration's response to...,2020-03-01,https://www.cnn.com/2020/03/01/politics/joe-bi...
17,Trump is expected to visit the CDC and NIH thi...,2020-03-01,https://www.cnn.com/2020/03/01/politics/donald...
19,'SNL' spoofs Mike Pence's new role leading cor...,2020-03-01,https://www.cnn.com/2020/03/01/media/snl-mike-...
...,...,...,...
33654,"Dow 30,000: How the market rally defied a pand...",2020-11-25,https://www.cnn.com/2020/11/25/investing/prema...
33655,Fauci says he's worried about what coronavirus...,2020-11-25,https://www.cnn.com/2020/11/25/health/us-coron...
33661,What you need to know about coronavirus on Wed...,2020-11-25,https://www.cnn.com/2020/11/25/world/coronavir...
33668,50 ways to celebrate a birthday during a pandemic,2020-11-25,https://www.cnn.com/2020/11/25/health/how-to-c...


In [5]:
articles = []
for i, row in covid_articles_meta[:10].iterrows():
    url = row.url
    
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    
    links = [a["href"] for a in soup.select(".zn-body__paragraph a")]

    content = soup.find_all(class_="zn-body__paragraph")
    article_text = " ".join([c.text for c in content])
    
    article = {
        "title": row.title,
        "date": row.date,
        "url": row.url,
        "links": links,
        "text": article_text,
    }
    articles.append(article)

In [6]:
all_links = [a["links"] for a in articles]
all_links = [a for b in all_links for a in b]
all_links

['https://www.cnn.com/videos/politics/2020/03/01/mike-pence-coronavirus-don-jr-fox-news-jake-tapper-sotu-vpx.cnn',
 'https://www.cnn.com/2020/03/01/politics/joe-biden-coronavirus-response-trump-administration-cnntv/index.html',
 'https://www.cnn.com/2020/02/28/politics/donald-trump-jr-coronavirus-democrats/index.html',
 'https://www.cnn.com/2020/02/26/politics/trump-coronavirus-stock-market/index.html',
 'https://twitter.com/tedlieu/status/1234152146611818497?s=20',
 'https://www.cnn.com/asia/live-news/coronavirus-outbreak-02-29-20-intl-hnk/h_b2418af32092b854e3591e11e08f2896',
 'https://www.cnn.com/travel/article/private-tour-louvre-paris/index.html',
 'https://www.who.int/emergencies/diseases/novel-coronavirus-2019/advice-for-public',
 'https://www.cnn.com/style/article/whats-so-special-about-the-mona-lisa/index.html',
 'https://www.cnn.com/travel/article/private-tour-louvre-paris/index.html',
 'https://www.cnn.com/asia/live-news/coronavirus-outbreak-03-01-20-intl-hnk/h_fab395d1124a91

In [7]:
cdc_counts = [a["text"].count("CDC") for a in articles]
cdc_counts

[0, 0, 3, 4, 0, 2, 1, 0, 1, 0]

In [8]:
who_counts = [a["text"].count("WHO") for a in articles]
who_counts

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [9]:
articles_df = pd.DataFrame(articles)
articles_df.to_csv("data/cnn_covid_articles_content.csv", index=False)